In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow import keras
from numpy import zeros, newaxis
from scipy import sparse
from scipy.sparse.linalg import spsolve
from joblib import Parallel,delayed
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from keras import utils as np_utils                     
from sklearn.model_selection import train_test_split  
from keras.layers import Dense,Flatten,Conv1D,MaxPooling1D,Dropout,BatchNormalization
from tensorflow.keras import datasets,layers,models
from keras.models import Sequential
from numpy import unique
from sklearn.metrics import mean_squared_error,r2_score
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import SMOTE
from keras import backend as K 
from keras.regularizers import l2
from keras.models import load_model
from keras.models import Model
from sklearn.utils import shuffle
from numpy.random import RandomState

np.random.seed(0)

def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

# load in CNN
weightsfile ='CNNBinaryFinalmodel.h5'
modelfile = 'CNNBinaryFinalmodel.json'

# load model from json
json_file = open(modelfile, 'r')
loaded_model_json = json_file.read()
json_file.close()
base_model = keras.models.model_from_json(loaded_model_json)
base_model.load_weights(weightsfile)
print(base_model.summary())
# remove the last 2 dense FC layers and freeze it
base_model.pop()
base_model.pop()
base_model.trainable = False

inp = base_model.input

last = base_model.layers[-1].output
x = Dense(1, activation='sigmoid', name='predictions')(last)

model2 = Model(inp, x)

model2.layers[1].trainable = True
model2.layers[2].trainable = True
model2.layers[3].trainable = True
model2.layers[4].trainable = True
model2.layers[5].trainable = True
model2.layers[6].trainable = True
model2.layers[7].trainable = False
model2.layers[8].trainable = False
model2.layers[9].trainable = False
model2.layers[10].trainable = False
model2.layers[11].trainable = False
model2.layers[12].trainable = False
model2.layers[13].trainable = False
model2.layers[14].trainable = False

print(model2.summary())

fnames=sorted(glob('Data_Pathway/*'))
files=[np.loadtxt(f) for f in fnames]
map=[f[:,3] for f in files]
map=[np.reshape(m,(-1,1011)) for m in map]
map_full = np.concatenate([m[:,:] for m in map],axis=0)

y = np.concatenate([np.repeat(i, len(m)) for i, m in enumerate(map)], axis=0)
map_full= savgol_filter(map_full, 11, 3, axis=0)
back = Parallel(n_jobs=8)(delayed(baseline_als)(j, 100000, 0.001) for j in map_full)
map_full= np.subtract(map_full, back)
map_full= np.reshape(map_full, (-1, 1011))
divide=map_full[:,999:1000]
X=map_full/divide

x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=0.5,stratify=y,shuffle=True)
x_val, x_test, y_val, y_test=train_test_split(x_test,y_test,test_size=0.8,stratify=y_test,shuffle=True)

pca = PCA(n_components=22)
pca.fit(x_train)
x_train= pca.transform(x_train)
x_val= pca.transform(x_val)
x_test= pca.transform(x_test)

x_train_cnn= np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_val_cnn= np.reshape(x_val, (x_val.shape[0], x_val.shape[1],1))
x_test_cnn= np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

base_learning_rate = 0.001
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
finetune_history=model2.fit(x_train_cnn, y_train,epochs=500,validation_data=(x_val_cnn,y_val))

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(hist['epoch'], hist['accuracy'],
           label='Train Accuracy')
    plt.plot(hist['epoch'], hist['val_accuracy'],
           label = 'Val Accuracy')
    plt.ylim([0,1.1])
    plt.legend()
    plt.show()

plot_history(finetune_history)

In [ ]:
fname=sorted(glob('"Unknown"Data_Pathway/*'))
file=[np.loadtxt(f) for f in fname]
map=[f[:,3] for f in file]
map=[np.reshape(m,(-1,1011)) for m in map]
map_full= np.concatenate([m[:,:] for m in map],axis=0)
map_full=np.array(map_full)
Y_test=np.repeat(0,400)
map_full= savgol_filter(map_full, 11, 3, axis=1)
back = Parallel(n_jobs=8)(delayed(baseline_als)(j, 100000, 0.001) for j in map_full)
map_full= np.subtract(map_full, back)
map_full= np.reshape(map_full, (-1, 1011))
divide=map_full[:,999:1000]
X_test=map_full/divide

print(X_test.shape)
print(Y_test.shape)

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,  "{:0.2f}".format(cm[i, j]),
                 horizontalalignment="center",fontsize=20,
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

X_cleaned= pca.transform(X_cleaned)
X_cleaned_cnn= np.reshape(X_cleaned, (X_cleaned.shape[0], X_cleaned.shape[1],1))
y_pred1 = (model2.predict(X_cleaned_cnn) > 0.5).astype("int32")

cn_matrix = confusion_matrix(y_cleaned,y_pred1)
np.set_printoptions(precision=3)

class_names=['below','above']
plot_confusion_matrix(cn_matrix, classes=class_names, normalize=True)